In [1]:
import redis
import math

In [2]:
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, charset="utf-8", decode_responses=True)

In [3]:
### based on https://github.com/RedisLabs/redis-recommend/blob/master/redrec/engine.go
### redis commands: https://redis.io/commands/zunionstore
def rate(item, user, score):
    redis_client.execute_command('ZADD', f'user:{user}:items', score, item)
    redis_client.execute_command('ZADD', f'item:{item}:scores', score, user)
    redis_client.execute_command('SADD', 'users', user)

In [4]:
def get_user_suggestions(user, max):
    return redis_client.execute_command('ZREVRANGE', f'user:{user}:suggestions', 0, max, 'WITHSCORES')

In [5]:
def batch_update_similar_items(max):
    users = redis_client.execute_command("SMEMBERS", "users")
    for user in users:
        candidates = get_similarity_candidates(user, max)
        #print(candidates)
        
        args = []
        args.append(f'user:{user}:similars')
        for candidate in candidates:
            if candidate != user:
                score = calculate_similarity(user, candidate)
                args.append(score)
                args.append(candidate)
                
        #print(*args)
        redis_client.execute_command('ZADD', *args) ## instead of args... in Go 
    
#batch_update_similar_items(100)

In [9]:
def get_suggest_candidates(user, max):
    similar_users = redis_client.execute_command('ZRANGE', f'user:{user}:similars', 0, max)
    max = len(similar_users)
    args = []
    args.append('ztmp')
    args.append(max+1)
    args.append(f'user:{user}:items')
    
    weights = []
    weights.append('WEIGHTS')
    weights.append(-1)
    #print(similar_users)
    for simuser in similar_users:
        args.append(f'user:{simuser}:items')
        weights.append(1)
    
    args = args + weights
    args.append('AGGREGATE')
    args.append('MIN')
    #print(*args)    
    redis_client.execute_command('ZUNIONSTORE', *args)
    candidates = redis_client.execute_command("ZRANGEBYSCORE", "ztmp", 0, "inf")
    ##TODO: candidates are not returned properly when user=1
    ##  ZUNIONSTORE ztmp 3 user:1:items user:2:items user:3:items WEIGHTS -1 1 1 AGGREGATE MIN
    ##  ZRANGEBYSCORE ztmp  0 "inf"   ->> !!returns empty
    
    redis_client.execute_command('DEL', 'ztmp')
    return candidates

In [10]:
def calculate_item_probability(user, item):    
    redis_client.execute_command("ZINTERSTORE", "ztmp", 2,  f'user:{user}:similars', f'item:{item}:scores', "WEIGHTS", 0, 1) ## https://redis.io/commands/zinterstore
    
    scores = redis_client.execute_command('ZRANGE', 'ztmp', 0, -1, 'WITHSCORES')
    #print(scores)
    redis_client.execute_command('DEL', 'ztmp')
    if len(scores) == 0: return 0

    score = 0
    for i in range(1, len(scores), 2):
        score += float(scores[i]) ## TODO: check this
    
    score /= float(len(scores) / 2.0)
    #print(score)
    return float(score)
calculate_item_probability(2, 4)

0

In [12]:
def update_suggested_items(user, max):
    items = get_suggest_candidates(user, max)
    
    ## TODO: newly added
    if len(items) == 0: return
    
    if max > len(items): max = len(items)
        
    args = []
    args.append(f'user:{user}:suggestions')
    for item in items:
        probability = calculate_item_probability(user, item)
        print(f'probability for user {user} and item {item} is {probability}')
        args.append(probability)
        args.append(item)
    print(*args)
    redis_client.execute_command('ZADD', *args)
    
update_suggested_items(1, 100)

probability for user 1 and item 5 is 2.0
probability for user 1 and item 3 is 2.0
probability for user 1 and item 4 is 3.0
user:1:suggestions 2.0 5 2.0 3 3.0 4


In [13]:
def get_user_items(user, max):
    return redis_client.execute_command('ZREVRANGE', f'user:{user}:items', 0, max)

In [14]:
def get_item_scores(item, max):
    return redis_client.execute_command('ZREVRANGE', f'user:{item}:scores', 0, max)

In [15]:
## checked (is fine)
def get_similarity_candidates(user, max):
    items = get_user_items(user, max)
    if max > len(items): max = len(items)
        
    args = []
    args.append('ztmp')
    args.append(max)
    for i in range(0, max):
        args.append(f'item:{items[i]}:scores')
    redis_client.execute_command('ZUNIONSTORE', *args)
    users = redis_client.execute_command('ZRANGE', 'ztmp', 0, -1)
    redis_client.execute_command('DEL', 'ztmp')
    
    return users

In [16]:
## TODO: not working properly
## NOTE: the higher the similarity the better (0:1)
## http://www.cs.carleton.edu/cs_comps/0607/recommend/recommender/itembased.html
def calculate_similarity(user, simuser):
    if user == simuser:
        return 1.0
    
    redis_client.execute_command("ZINTERSTORE", "ztmp", 2,  f'user:{user}:items', f'user:{simuser}:items', "WEIGHTS", 1, -1) ## https://redis.io/commands/zinterstore
    user_diffs = redis_client.execute_command('ZRANGE', 'ztmp', 0, -1, 'WITHSCORES')
    redis_client.execute_command('DEL', 'ztmp')
    #print(user_diffs)
    if len(user_diffs) == 0:
        return 0
    
    score = 0.0
    #print(f'user_diffs: {user_diffs}')
    ## RMS Error
    ## https://statweb.stanford.edu/~susan/courses/s60/split/node60.html
    
    ## WITHSCORE returns the score every each second element including the element itself
    ## for ex if user_diffs = ['2', '0'] then 2 is the similar item between 2 users and the score is 0 
    for i in range(1, len(user_diffs), 2):
#         print('inside')
#         print(user_diffs[i])
        diff_val = float(user_diffs[i])
        #print(f'diff_val: {diff_val}')
        score += math.pow(diff_val, 2)
    score /= float(len(user_diffs) / 2)
    score = math.sqrt(score)
    return score

calculate_similarity(1, 2)

3.0

In [17]:
####### PUBLIC METHODS #########
def suggest(user, max):
    update_suggested_items(user, max)
    
    suggestions_with_scores = get_user_suggestions(user, max)
    #print(f'####### suggestions_with_scores for user {user}: {suggestions_with_scores}')
    clean_suggestions = []
    for i in range(0, len(suggestions_with_scores), 2):
        #print(f'clean_suggestions: {suggestions_with_scores[i]}')
        clean_suggestions.append({
            'item': suggestions_with_scores[i],
            'score': suggestions_with_scores[i+1]
        })
    print(f'####### suggestions for user {user}: {clean_suggestions}')

def update(max):
    batch_update_similar_items(max)

def get_probability(user, item):
    print(f'####### get_probability: {user} {item}')
    score = calculate_item_probability(user, item)

def rate_item(user, item, score):
    print(f'####### rate_items: user {user} rated item {item} with score {score}')
    rate(item, user, score)
    

In [18]:
###### EXAMPLE #######
redis_client.flushall()
### rate_item(user, item, score)
rate_item(2, 2, 2) 
rate_item(2, 3, 2) 
rate_item(2, 4, 3) 
rate_item(2, 5, 1) 

rate_item(1, 1, 3)
rate_item(1, 2, 5) 
rate_item(3, 2, 3) 
rate_item(3, 5, 3) 

update(100)
suggest(1, 100) ## working somehow ?
suggest(2, 100) ## working somehow ?
suggest(3, 100) ## working somehow

####### rate_items: user 2 rated item 2 with score 2
####### rate_items: user 2 rated item 3 with score 2
####### rate_items: user 2 rated item 4 with score 3
####### rate_items: user 2 rated item 5 with score 1
####### rate_items: user 1 rated item 1 with score 3
####### rate_items: user 1 rated item 2 with score 5
####### rate_items: user 3 rated item 2 with score 3
####### rate_items: user 3 rated item 5 with score 3
probability for user 1 and item 5 is 2.0
probability for user 1 and item 3 is 2.0
probability for user 1 and item 4 is 3.0
user:1:suggestions 2.0 5 2.0 3 3.0 4
####### suggestions for user 1: [{'item': '4', 'score': '3'}, {'item': '5', 'score': '2'}, {'item': '3', 'score': '2'}]
probability for user 2 and item 1 is 3.0
user:2:suggestions 3.0 1
####### suggestions for user 2: [{'item': '1', 'score': '3'}]
probability for user 3 and item 3 is 2.0
probability for user 3 and item 1 is 3.0
probability for user 3 and item 4 is 3.0
user:3:suggestions 2.0 3 3.0 1 3.0 4
####### 

In [19]:
## building the API for the recommender above
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}
